# Pseudonymization Tool

Contexte : rendre les contrats confidentiels avant de les transmettre à une IA juridique dont le traitement est basé sur le Cloud.

Cet outil de pseudonymization est conçu pour remplacer les noms de personnes et d'organisations dans un texte tout en respectant certaines règles, comme ne pas pseudonymiser les figures historiques, les personnages fictifs ou les authorités publiques.


# Load the dataset

In [ ]:
# visionner nos données
df = pd.read_json('ner_tech_test_dataset.jsonl', lines = True)
df.head()

,text,spans
0,A county in Glarus named Humberside recently p...,"[{'start': 272, 'end': 281, 'label': 'PERSON_N..."
1,▪ Changesinsocietalexpectationsandregulatoryre...,"[{'start': 361, 'end': 370, 'label': 'ORGANIZA..."
2,"15.Ce méme 20 septembre 2006, la société Solva...","[{'start': 41, 'end': 56, 'label': 'ORGANIZATI..."
3,c) Physical inventory counts are to be perf...,"[{'start': 478, 'end': 481, 'label': 'ORGANIZA..."
4,legal group plans suit to void california powe...,"[{'start': 0, 'end': 11, 'label': 'ORGANIZATIO..."


# Process dataset

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Fonction pour créer un dictionnaire des labels
def create_label_mappings():
    labels = ["O", "B-PERSON_NAME", "I-PERSON_NAME", "B-ORGANIZATION_NAME", "I-ORGANIZATION_NAME", "B-AUTHORITY_NAME", "I-AUTHORITY_NAME"]
    label2id = {label: idx for idx, label in enumerate(labels)}
    id2label = {idx: label for idx, label in enumerate(labels)}
    return label2id, id2label

# Convertir labels en ids
def convert_labels_to_ids(example, label2id):
    example["labels"] = [label2id[label] for label in example["labels"]]
    return example

# Convertir ids en labels
def convert_ids_to_labels(example, id2label):
    example["labels"] = [id2label[label_id] for label_id in example["labels"]]
    return example

label2id, id2label = create_label_mappings()

# Fonction de prétraitement des données (avec conversion des labels en ids)
def preprocess_data_with_ids(example):
    '''example: un dictionnaire dont les clés sont 'text' et 'spans', pour un texte.'''
    tokens = tokenizer(example["text"], truncation=True, return_offsets_mapping=True)
    labels = ["O"] * len(tokens["input_ids"])
    for span in example["spans"]:
        for idx, (start, end) in enumerate(tokens["offset_mapping"]):
            if start >= span["start"] and end <= span["end"]:  # Vérification stricte des limites
                if start == span["start"]:
                    labels[idx] = f"B-{span['label']}"
                else:
                    labels[idx] = f"I-{span['label']}"
    tokens["labels"] = labels
    tokens.pop("offset_mapping")  # Supprime les offsets pour alléger la sortie
    return convert_labels_to_ids(tokens, label2id)

# Charger les données JSONL
data = [{"text": row["text"], "spans": row["spans"]} for row in pd.read_json("ner_tech_test_dataset.jsonl", lines=True).to_dict(orient="records")]

# Convertir en Dataset HuggingFace
dataset = Dataset.from_list(data)

# Prétraiter les données
processed_dataset = dataset.map(preprocess_data_with_ids, batched=False)

# Diviser en ensembles d'entraînement, validation et test
train_test_split = processed_dataset.train_test_split(test_size=0.2, seed=42)
train_val_split = train_test_split["train"].train_test_split(test_size=0.1, seed=42)

train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = train_test_split["test"]

# Résumé
print(f"Entraînement : {len(train_dataset)} exemples")
print(f"Validation : {len(val_dataset)} exemples")
print(f"Test : {len(test_dataset)} exemples")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/38122 [00:00<?, ? examples/s]

Entraînement : 27447 exemples
Validation : 3050 exemples
Test : 7625 exemples


In [ ]:
processed_dataset

Dataset({
    features: ['text', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 38122
})

Notes:

labels : étiquettes IOB convertis en indices numériques.

token_type_ids : indicateurs pour distinguer différentes parties du texte lorsque le modèle traite plusieurs segments (par exemple Q/A, où tous les subwords de la question sont 0, et ceux de l'answer sont 1).\
Ici on a qu'un segment, donc tous les token_type_ids sont 0.\
(Générés automatiquement par le tokenizer. Pas exploités ici)

train_test_split : Divise les données en deux parties :
- 80% des données pour l'entraînement.
- 20% des données pour le test.

seed=42 : Assure que la division est reproductible.

Puis division supplémentaire de l'ensemble d'entraînement pour obtenir un ensemble de validation :

    train_val_split = train_test_split["train"].train_test_split(test_size=0.1, seed=42)

Prend 10% du train set pour le valid set.

A la fin, on a donc:

- train_dataset : Données pour entraîner le modèle (72% des données initiales).
- val_dataset : Données pour valider le modèle (8% des données initiales).
- test_dataset : Données pour tester le modèle (20% des données initiales).

# Evaluation avant le fine-tuning

In [ ]:
import torch
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label2id), label2id=label2id, id2label=id2label)

model.to(device)

def evaluate_model(model, dataset):
    model.eval()  # Passer le modèle en mode évaluation

    all_true_labels = []
    all_pred_labels = []

    for example in dataset:

        inputs = tokenizer(example["text"], truncation=True, padding=True, return_tensors="pt").to(device)

        # Propagation avant (inférence) pour obtenir les prédictions
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Récupérer les indices des prédictions
        preds = np.argmax(logits.detach().cpu().numpy(), axis=2)

        # Récupérer les labels réels
        true_labels = example["labels"]

        # Filtrer les prédictions et labels pour enlever les tokens de padding
        true_labels = [label for label in true_labels if label != -100]
        preds = [pred for pred, label in zip(preds[0], true_labels) if label != -100]

        all_true_labels.extend(true_labels)
        all_pred_labels.extend(preds)

    # Calculer les métriques : précision, rappel et f-mesure
    precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_pred_labels, average='weighted')

    return {"precision": precision, "recall": recall, "f1": f1}

# Évaluation sur l'ensemble de validation
results = evaluate_model(model, val_dataset)
print("Résultats d'évaluation avant fine-tuning:", results)

Résultats d'évaluation avant fine-tuning: {'precision': 0.8498823039500508, 'recall': 0.28026758369124877, 'f1': 0.40919325073870266}



# Fine-tuning

In [ ]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from torch.utils.data import DataLoader
import torch

# Charger le modèle avec le nombre de labels
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased", num_labels=len(label2id), id2label=id2label, label2id=label2id
)

# Fonction pour créer DataLoader
def create_dataloader(dataset, batch_size=16):
    data_collator = DataCollatorForTokenClassification(tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    return dataloader

# Créer les dataloaders pour l'entraînement et la validation
train_dataloader = create_dataloader(train_dataset)
val_dataloader = create_dataloader(val_dataset)

# Configuration des arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to = "none"
)

# Configuration du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Jeu de données d'entraînement
    eval_dataset=val_dataset,  # Jeu de données de validation
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer), # Gestion du padding et du batching
)

# Fine-tuning du modèle
trainer.train()

# Sauvegarde du modèle fine-tuné
trainer.save_model("./fine_tuned_bert")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-7e5d6c7583a1>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.060200,0.056796
2,0.040400,0.053775
3,0.038900,0.054206


(Note sur num_labels (dans les arguments du modèle) :

Définit le nombre total d'étiquettes (classes) que le modèle doit prédire.\
Il est calculé en prenant l'ensemble des labels présents dans train_dataset["labels"] (on garde l'étiquette 'O' pour la cohérence avec les id2labels et labels2ids).\
(C'est ['O','B-PERSON_NAME', 'I-PERSON_NAME', 'B-ORGANIZATION_NAME', 'I-ORGANIZATION_NAME']))

# Recharger le modèle finetuned

In [ ]:
# RECHARGER LE MODELE FINE-TUNE

from transformers import AutoModelForTokenClassification

finetuned_model = AutoModelForTokenClassification.from_pretrained("results_fine_tuned_bert") # fichier des paramètres du modèle fine-tuné, que je vous ai fourni

tokenizer = AutoTokenizer.from_pretrained("results_fine_tuned_bert")

In [ ]:
print(finetuned_model.config)

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "results_fine_tuned_bert",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PERSON_NAME",
    "2": "I-PERSON_NAME",
    "3": "B-ORGANIZATION_NAME",
    "4": "I-ORGANIZATION_NAME",
    "5": "B-AUTHORITY_NAME",
    "6": "I-AUTHORITY_NAME"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-AUTHORITY_NAME": 5,
    "B-ORGANIZATION_NAME": 3,
    "B-PERSON_NAME": 1,
    "I-AUTHORITY_NAME": 6,
    "I-ORGANIZATION_NAME": 4,
    "I-PERSON_NAME": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",

In [ ]:
# Tester avec un exemple d'entrée
from transformers import AutoTokenizer
import torch

# Exemple
example_text = "One time, John Smith worked at Coca Cola and he drank from the Supreme Court."

# Tokenizer l'entrée
inputs = tokenizer(example_text, return_tensors="pt")

# Effectuer une prédiction
finetuned_model.eval()  # Met le modèle en mode évaluation
with torch.no_grad():
    outputs = finetuned_model(**inputs)

# Obtenir les scores des labels pour chaque token
logits = outputs.logits
predicted_ids = logits.argmax(dim=-1)  # Obtenir l'ID du label avec le score le plus élevé

# Décoder les tokens et les prédictions
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
predicted_labels = [finetuned_model.config.id2label[label_id.item()] for label_id in predicted_ids[0]]

print("Tokens :", tokens)
print("Labels :", predicted_labels)


Tokens : ['[CLS]', 'One', 'time', ',', 'John', 'Smith', 'worked', 'at', 'Coca', 'Cola', 'and', 'he', 'drank', 'from', 'the', 'Supreme', 'Court', '.', '[SEP]']
Labels : ['O', 'O', 'O', 'O', 'B-PERSON_NAME', 'I-PERSON_NAME', 'O', 'O', 'B-ORGANIZATION_NAME', 'I-ORGANIZATION_NAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


# Evaluation du modèle fine-tuné

In [ ]:
from transformers import AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support
import torch
import numpy as np

# (déjà initialisés plus haut)
#tokenizer = AutoTokenizer.from_pretrained("results_fine_tuned_bert")
#finetuned_model = AutoModelForTokenClassification.from_pretrained("results_fine_tuned_bert")

finetuned_model.to(device)

results = evaluate_model(finetuned_model, val_dataset) # fonction evaluate_model définie plus haut
print("Résultats d'évaluation du modèle fine-tuné:", results)

Résultats d'évaluation du modèle fine-tuné: {'precision': 0.9826046957472239, 'recall': 0.9828965921605813, 'f1': 0.982715197383457}




Résultats d'évaluation du modèle fine-tuné: {'precision': 0.9826046957472239, 'recall': 0.9828965921605813, 'f1': 0.982715197383457}


Autre fonction d'évaluation (avec GPU):

In [ ]:
from sklearn.metrics import classification_report
import torch
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

finetuned_model.eval()

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_labels = labels[labels != -100]
    predicted_labels = predictions[labels != -100]

    true_labels_named = [id2label[label] for label in true_labels.tolist()]
    predicted_labels_named = [id2label[label] for label in predicted_labels.tolist()]

    report = classification_report(true_labels_named, predicted_labels_named, output_dict=True)

    # dictionary of metrics
    return {
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"],
        "accuracy": report["accuracy"]
    }

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",  # Effectuer l'évaluation à chaque époque
    disable_tqdm=False,
    metric_for_best_model="f1",
    report_to = "none"
)

trainer = Trainer(
    model=finetuned_model,
    args=training_args,
    data_collator=data_collator,  # Utiliser le data_collator pour la gestion des labels
    compute_metrics=compute_metrics,  # Calcul des métriques
    eval_dataset=test_dataset,  # Jeu de test
)

# Évaluation sur le jeu de test
eval_results = trainer.evaluate()

print("Résultats de l'évaluation :", eval_results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


eval_results = \
{'eval_loss': 0.054639969021081924,\
 'eval_model_preparation_time': 0.0054,\
 'eval_precision': 0.9823388085251653,\
 'eval_recall': 0.9826839249770052,\
 'eval_f1': 0.982455596035671,\
 'eval_accuracy': 0.9826839249770052,\
 'eval_runtime': 216.6362,\
 'eval_samples_per_second': 35.197,\
 'eval_steps_per_second': 2.202}

On a donc fine-tuné un modèle qui est capable de reconnaître les named entity. Toutes. Donc maintenant il faut trier ce que l'on pseudonymise et ce qu'on laisse tel quel.

# PSEUDONYMISATION

In [ ]:
!pip install requests

In [ ]:
from collections import defaultdict
from transformers import pipeline
import requests

# Pipeline de prédiction à partir du modèle fine-tuné
ner_pipeline = pipeline("ner", model=finetuned_model, tokenizer=tokenizer, aggregation_strategy="simple")

def historical_fictional(name):
    """
    Vérifie si le nom correspond à une figure historique ou un personnage fictif en interrogeant Wikidata.
    """
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={name}&language=fr&limit=1&format=json" # API de Wikidata
    response = requests.get(url).json()
    if not response['search']:
        return False
    return True

def pseudonymize_text(text):

    pseudonym_dict = {
        "PERSON_NAME": {},
        "ORGANIZATION_NAME": {},
    }

    predictions = ner_pipeline(text)
    replaced_text = text  # Texte initial avant modification
    offset = 0  # Décalage cumulé après chaque remplacement

    for pred in sorted(predictions, key=lambda x: x['start']):  # Assurer un traitement de gauche à droite
        entity_type = pred["entity_group"]
        original_entity = pred["word"]

        # Vérifier si l'entité est une personne historique/fiction
        if entity_type == "PERSON_NAME" and historical_fictional(original_entity):
            continue  # Ne pas pseudonymiser cette entité

        if entity_type in pseudonym_dict:  # Seules PERSON_NAME et ORGANIZATION_NAME sont pseudonymisées
            if original_entity not in pseudonym_dict[entity_type]:
                pseudonym_dict[entity_type][original_entity] = f"[{entity_type}_{len(pseudonym_dict[entity_type]) + 1}]"
            placeholder = pseudonym_dict[entity_type][original_entity]

            # Calculer les indices ajustés en tenant compte du décalage
            start_idx = pred["start"] + offset
            end_idx = pred["end"] + offset

            # Remplacement
            replaced_text = replaced_text[:start_idx] + placeholder + replaced_text[end_idx:]

            # Mettre à jour le décalage
            offset += len(placeholder) - (end_idx - start_idx)

    output_file_path = "output_pseudonymized.txt"

    # Ecrire le texte pseudonymisé dans le fichier de sortie
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(replaced_text)

    print(f"Pseudonymized text written to {output_file_path}")
    return replaced_text


In [ ]:
# Exemple
test_text = """
Faustine Laroche, CEO of Gatewai SAS, met with officials from the Supreme Court.
She discussed plans with César Dupont from Coca Cola about a possible merger.
Then, Faustine Laroche went home. And Coca Cola went down.
Albert Einstein got a call from Robert Oppenheimer.
"""

# Pseudonymisation
pseudonymized_text = pseudonymize_text(test_text)
print("Original text:", test_text)
print("Pseudonymized text:", pseudonymized_text)


Original text: 
Faustine Laroche, CEO of Gatewai SAS, met with officials from the Supreme Court.
She discussed plans with César Dupont from Coca Cola about a possible merger.
Then, Faustine Laroche went home. And Coca Cola went down.
Albert Einstein got a call from Robert Oppenheimer.

Pseudonymized text: 
[PERSON_NAME_1], CEO of [ORGANIZATION_NAME_1], met with officials from the Supreme Court.
She discussed plans with [PERSON_NAME_2] from [ORGANIZATION_NAME_2] about a possible merger.
Then, [PERSON_NAME_1] went home. And [ORGANIZATION_NAME_2] went down.
Albert Einstein got a call from Robert Oppenheimer.



Analyses dans le README.
